In [ ]:
# 查看环境安装情况
!pip freeze | grep tensor

In [ ]:
# 查看文件行数
!wc -l data/poetry/*.txt

In [ ]:
# 查看文件内容
!head -5 data/poetry/*.txt

In [ ]:
# 多文件操作
with open('data/poetry/raw.txt', 'r') as rawfp,\
  open('data/poetry/input.txt', 'w') as infp,\
  open('data/poetry/output.txt', 'w') as outfp:
    
    prev_line = ''
    for curr_line in rawfp:
        curr_line = curr_line.strip()
        # poems break at empty lines, so this ensures we train only
        # on lines of the same poem
        if len(prev_line) > 0 and len(curr_line) > 0:       
            infp.write(prev_line + '\n')
            outfp.write(curr_line + '\n')
        prev_line = curr_line

In [ ]:
#github忽略文件
touch .gitignore
vi .gitignore
# 忽略文件夹
/doc/

In [ ]:
# 合并checkpoints
python avg_checkpoints.py --checkpoints=[tacotron_model.ckpt-345000,tacotron_model.ckpt-350000] --output_path="/Users/liuxinxin/Downloads/new.ckpt"